###Project 2: Janine Quigley and Jon Adamson
#### For this project, our team is interested in reviewing and analyzing political data from the 116th United States House of Representatives. Our central question is: What political factors have an effect on the progress of a bill in the House of Representatives?
#### To answer this question, we are websraping from *clerk.house.gov* to look at specific bills and the voting breakdown of yes/no votes on that bill. Additionally, we are gathering the House member name, party, and state.

Data Source: https://clerk.house.gov/evs/2018/roll023.xml with roll number changing depending on what bill is being scraped.

#### Step 1: Import Libraries

In [90]:
import pandas as pd
import requests
from lxml import etree
import io
import random
import plotly.express as px

#### Step 2: Create Lists of Lists from .xml scraping

In [91]:
def reader(bill):
  ''' 
  Reads xml links, checks response status code, and returns the roots of the tree
  Inputs: Number of bill whose vote data will be scraped
  Outputs: Returns the root of the tree
  
  '''

  xml = f'https://clerk.house.gov/evs/2019/roll{bill}.xml' #bill number must be 3 digits
  response = requests.get(xml)

  #assert response.status_code == 200

  response.content
  htmlparser = etree.HTMLParser()
  tree1 = etree.parse(io.BytesIO(response.content), htmlparser)
  root = tree1.getroot()
  return root


def legnum(root):
    legisnumber = root.xpath('.//legis-num/text()')
    return legisnumber

def varlist(root,var):
  '''
  Uses xpath notation to return the 'vote-data' element, whose children contain the data we want to scrape.
  Inputs: root: root that is returned from the reader(bill) function
          var: the rest of the xpath phrase that gaqthers information on the intended variable
  Outputs: The first item in the vote-data list with the children needed to get the rest of the data

  '''
  votedata = root.xpath('.//vote-data')
  votedata = votedata[0].xpath(f"./recorded-vote/{var}") # use var to choose which variable to grab

  return votedata


def listoflists(root):
  ''' 
  Creates a list of lists with the bill's House members, member state, member party, and member vote.
  Inputs: None
  Outputs: List of lists
  '''
  states = varlist(root,'legislator/@state')
  names = varlist(root,'legislator/@unaccented-name')
  party = varlist(root,'legislator/@party')
  vote = varlist(root,'vote/text()')
  data = [] #empty list
  data.append(names) # appending to make LoL
  data.append(states)
  data.append(party)
  data.append(vote)
  return data

#### Step 3: Create Dataframe from Lists of Lists

In [92]:
def dataframe(lol):
  ''' 
  Creates a datafrom from a list of lists.
  Inputs: lol: a list of lists
  Outputs: pandas dataframe
  '''
  df = pd.DataFrame(lol)
  df = df.T
  df.set_axis(['Name','State','Party','Vote'],axis='columns',inplace=True) # setting axis names
  return df


In [93]:
data116 = pd.read_csv('house_legislation_116.csv')

#### We plan on using these dataframes in combination with the rest of our data to answer our central question. This data provides us with some of the political factors we believe will affect the progress of a bill. We can use these dataframes to make visualizations to compare the member party versus the yes/no distribution of votes. Also, these dataframes give us the opportunity to run statistical tests to see if one party is more likely to vote yes or no based on the policy area.

In [94]:
import sqlite3 as sql
def sqlcreate(db,cqry,num):
  '''
  Establishes a connection to a sql database
  Input: db, cqry, num
  Output: None
  '''
  connection = sql.connect(db)
  cursor = connection.cursor()
  cursor.execute(cqry)

In [95]:
cqry = '''
CREATE TABLE votetable(
  Name VARCHAR(100) NOT NULL,
  State VARCHAR(100) NOT NULL,
  Party VARCHAR(100) NOT NULL,
  Vote VARCHAR(100) NOT NULL,
  PRIMARY KEY (Name)
  )
  '''

In [96]:
def lolcreation(df):
  '''
  Takes a dataframe and returns a list of tuples
  Input: dataframe
  Output: s, list of tuples
  '''
  lol = []
  for i in range(len(df)):
    a = list(df.iloc[i])
    lol.append(tuple(a))
  s = ''
  for row in lol[:-1]:
    s += str(row) + ', '
  s = s+ str(lol[-1])
  return s


def insertdb(df,db,iqry,s):
  '''
  Inserts data into database
  Input: df, db, iqry, s
  Output: None
  '''
  connection = sql.connect(db)
  cursor = connection.cursor()
  cursor.execute(iqry)
  connection.commit()


In [97]:
def sqlselect(db,sqry):
  '''
  
  '''
  connection = sql.connect(db)
  cursor = connection.cursor()
  result = cursor.execute(sqry)
  return result.fetchall()

In [98]:
def loop(numb):
  '''
  Creates a list of random numbers that will be used to read and scrape bill information
  Input: numb, a number
  output: billnumber, list of random numbers
  '''
  bills = []
  billnumber = []
  for i in range(numb):
    bills.append(random.randint(1,700))
  for item in bills:
    if item < 100:
      item = "0" + str(item)
    elif item < 10:
      item = '00' + str(item)
    billnumber.append(item)
  return billnumber


def loop2(root,billnumber):
  '''
  Uses reader function and appends the HR number of the bill to a list
  Input: root, billnumber
  Output: HRnum, list containing the HR numbers of the bills we are using

  '''
  HR = []
  HRnum = []
  for item in billnumber:
    root = reader(item)
    number = legnum(root)
    HR.append(number)
    for item in HR:
      if len(item) == 1:
        item = item[0].replace(' ','.')
        HRnum.append(item)
    return HRnum

def loopinfo(HRnum):
  '''
  Iterates through HRnum list and extracts the specific bill information.
  This information is appended to a dictionary that contains policy area of the bill and bill progress.
  Input: HRnum
  Output: abdict, dictionary. 
  '''
  abdict = {'Area':[],'Status':[]}
  for item in HRnum:
    a = data116.loc[data116['bill_id'] == item]['policy_area']
    for obj in a:
      abdict['Area'].append(obj)
  for item in HRnum:
    b = data116.loc[data116['bill_id'] == item]['bill_progress']
    for obj in b:
      abdict['Status'].append(obj)
  return abdict


In [99]:
db = 'test1.db'
def finalpopulate(n):
  '''
  Takes n(number) and uses an SQL statement that creates n empty tables in our database, each corresponding to a bill. 
  The tables are then populating with the data of each respective bill using an SQL statement.
  Inputs: n, a number
  Outputs: 
  '''
  billnumber = ['043', 367, 487, 674, 234, 345, 341, 352, 525, 191]
  for item in billnumber:
    root = reader(item)
    lol = listoflists(root)
    df = dataframe(lol)
    HRnum = loop2(root,billnumber)
    cqry = f'''
CREATE TABLE votetable{item}(
  Name VARCHAR(100) NOT NULL,
  State VARCHAR(100) NOT NULL,
  Party VARCHAR(100) NOT NULL,
  Vote VARCHAR(100) NOT NULL,
  PRIMARY KEY (Name)
  )
  '''
    sqlcreate(db,cqry,n)
    s = lolcreation(df)
    iqry = f'''INSERT INTO votetable{item} VALUES ''' + s
    insertdb(df,db,iqry,s)
finalpopulate(10)